# Paleogenomics paper section

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

# Load regulatory islands data

**Genes in regulatory islands**

In [ ]:
regisl = pd.read_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/hg38_regulatory_islands_genenames.tsv", sep='\t')

In [ ]:
regisl_genenames = regisl['gene_name'].str.split(", ").explode().drop_duplicates()

regisl_genenames.dropna(inplace=True)

regisl_genenames = regisl_genenames.tolist()

In [ ]:
len(set(regisl_genenames))

**Regulatory islands genes in Deserts**

In [ ]:
regisl_deserts = pd.read_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/hg38_regulatory_islands_DESERTS.tsv", sep='\t')

In [ ]:
ri_deserts_genenames = regisl_deserts['gene_name'].str.split(", ").explode().drop_duplicates()

In [ ]:
ri_deserts_genenames.dropna(inplace=True)

In [ ]:
ri_deserts_genenames = ri_deserts_genenames.tolist()

**Regulatory islands genes in Positive Selection**

In [ ]:
regisl_possel = pd.read_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/regulatory_islands/hg38_regulatory_islands_POS_Sel.tsv", sep='\t')

In [ ]:
ri_possel_genenames = regisl_possel['gene_name'].str.split(", ").explode().drop_duplicates()

In [ ]:
ri_possel_genenames.dropna(inplace=True)

In [ ]:
ri_possel_genenames = ri_possel_genenames.tolist()

# Overlap with Moriano & Boeckx, 2020

In [ ]:
#Download BMC data
!wget https://figshare.com/ndownloader/files/20997426 -O tmp_bmc_genes.xlsx
    
xls = pd.ExcelFile('tmp_bmc_genes.xlsx')
bmc1 = pd.read_excel(xls, 'S1_ENH_onlyhuman_5to20')
bmc2 = pd.read_excel(xls, 'S2_PROM_onlyhuman_5to20')

bmc_genes = []
bmc_genes.append(bmc1['gene_name'])
bmc_genes.append(bmc2['gene_name'])

list_BMC = [item for sublist in bmc_genes for item in sublist]

list_BMC = [*set(list_BMC)]

print(len(list_BMC))

!rm tmp_bmc_genes.xlsx

**Regulatory islands genes in Moriano & Boeckx, 2020**

In [ ]:
overlap_w_BMC = list(set(list_BMC) & set(regisl_genenames))

print("% Overlap gene names: "+str(len(set(overlap_w_BMC))/len(list_BMC)*100))

In [ ]:
rr_hg19 = pd.read_csv("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/regulatory_islands_completeINFO.tsv", sep='\t')

In [ ]:
all_pos_bmc = pd.concat([bmc1['SNC_coordinate'], bmc2['SNC_coordinate']])

In [ ]:
all_pos_bmc = 'chr'+all_pos_bmc

In [ ]:
allpos_rrhg19 = rr_hg19['Chr'].astype(str)+":"+rr_hg19['SNV_end_hg19'].astype(str)

In [ ]:
len(set(all_pos_bmc) & set(allpos_rrhg19))

In [ ]:
len(set(all_pos_bmc) & set(allpos_rrhg19))/len(set(all_pos_bmc))*100

# Regulatory islands genes that are present in the 4K HVGs

In [ ]:
import celloracle as co

In [ ]:
tr21_pcw16 = co.load_hdf5("/home/jovyan/jm_jlab/data_indNeuro/1.GRN/tr21_pcw16.celloracle.oracle")

In [ ]:
tr21_pcw16.adata.var_names

In [ ]:
regisl_genenames_in4K = list(set(tr21_pcw16.adata.var_names) & set(regisl_genenames))

In [ ]:
len(regisl_genenames_in4K)

In [ ]:
ri_deserts_in4K = list(set(tr21_pcw16.adata.var_names) & set(ri_deserts_genenames))

In [ ]:
ri_possel_in4K = list(set(tr21_pcw16.adata.var_names) & set(ri_possel_genenames))

# Top 100 genes in piNMF - Trevino

**oRG**

In [ ]:
tr21_dir = '/home/jovyan/jm_jlab/data_indNeuro/3.NMF_data/4k_oRG_pcw16/'

In [ ]:
topgenes_oRG = pd.read_csv(tr21_dir+'4k_oRG_pcw16modules_to_oRG_topgenes_pcw16_4K_piNMF.tsv', sep='\t')

In [ ]:
topgenes_oRG.columns

In [ ]:
tmp_early = list(set(topgenes_oRG['Module_1']) | set(topgenes_oRG['Module_2']))

In [ ]:
len(tmp_early)

In [ ]:
tmp_late = list(set(topgenes_oRG['Module_3']) | set(topgenes_oRG['Module_4']))

In [ ]:
len(tmp_late)

**oRG Early vs Late modules** in regulatory islands

In [ ]:
islands_info = pd.read_csv("/home/jovyan/jm_jlab/data_CBL/indNeuro_tmp/regulatory_region_data/regulatory_region_results/regulatory_islands_completeINFO.tsv", sep='\t')

islands_info2 = islands_info.assign(tmp_genename=islands_info['gene_name'].str.split(', ')).explode('tmp_genename')

Early_Risland = islands_info2[islands_info2['tmp_genename'].isin(tmp_early)].drop(columns='tmp_genename')

Late_Risland = islands_info2[islands_info2['tmp_genename'].isin(tmp_late)].drop(columns='tmp_genename')

Early_Risland.reset_index(drop=True, inplace=True)

Late_Risland.reset_index(drop=True, inplace=True)

In [ ]:
for i in topgenes_oRG.columns:
    print(len(list(set(regisl_genenames_in4K) & set(topgenes_oRG[i]))))

In [ ]:
tr21_dir_ipc = '/home/jovyan/jm_jlab/data_indNeuro/3.NMF_data/4k_IPC_pcw16/4k_IPC_pcw16/'

In [ ]:
topgenes_IPC = pd.read_csv(tr21_dir_ipc+'modules_to_IPC_topgenes_pcw16_4K_piNMF.tsv', sep='\t')

In [ ]:
for i in topgenes_IPC.columns:
    print(len(list(set(regisl_genenames_in4K) & set(topgenes_IPC[i]))))

In [ ]:
tmp_early_ipc = list(set(topgenes_IPC['Module_1']) | set(topgenes_IPC['Module_2']))

In [ ]:
len(tmp_early_ipc)

In [ ]:
tmp_late_ipc = list(set(topgenes_IPC['Module_3']) | set(topgenes_IPC['Module_4']))

In [ ]:
len(tmp_late_ipc)

In [ ]:
set(tmp_early_ipc) & set(tmp_late_ipc)

In [ ]:
len(set(regisl_genenames_in4K) & set(tmp_early_ipc))

In [ ]:
len(set(regisl_genenames_in4K) & set(tmp_late_ipc))

In [ ]:
for i in [tmp_early, tmp_late]:
    print(len(set(regisl_genenames_in4K) & set(i)) / len(set(i))*100)

In [ ]:
def p_EarlyLate_fct(list_of_genes, topgenes_NMF, branch_name, database_name):
    
    """
    Calculate percentage of overlap between an input list of genes (e.g. genes found in deserts of introgression) and
    genes found in piNMF modules. Modules 1 and 2 are grouped as 'Early', and Modules 3 and 4 as 'Late'. 
    
    
    Parameters
    ----------
        

    list_of_genes: Evolutionary-relevant dataset
    topgenes_NMF: 100 marker genes from pseudotime-informed NMF analysis
    branch_name: vRG-to-IPC or vRG-to-IPC in our case, for instance. 
    database_name: Define list_of_genes
      
    Returns
    -------
    
    dataframe
    """
    
    overlap_Early = []
    overlap_Late = []
    Early = []
    Late = []
    module1_2 = set(topgenes_NMF['Module_1']) | set(topgenes_NMF['Module_2'])
    module3_4 = set(topgenes_NMF['Module_3']) | set(topgenes_NMF['Module_4'])
    
    for i in topgenes_NMF.columns:
        if i == 'Module_1' or i == 'Module_2':
            overlap_genes = list(set(list_of_genes) & module1_2)
            Early.extend([gene for gene in overlap_genes if gene not in Early])
            
            overlap_Early.append(len(set(Early))/(len(module1_2))*100)
            
            
        elif i == 'Module_3' or i == 'Module_4':
            overlap_genes = list(set(list_of_genes) & module3_4)
            Late.extend([gene for gene in overlap_genes if gene not in Late])
            
            overlap_Late.append(len(set(Late))/(len(module3_4))*100)
      
    d = {'Branch': [branch_name]*2, 
         'Database': [database_name]*2, 
         'Module': ['Early', 'Late'], 
         'Genes': [",".join(Early), ",".join(Late)],
         'Percentage overlap': [overlap_Early[0], overlap_Late[0]]}
    return pd.DataFrame(d)


In [ ]:
ipc_overlap = p_EarlyLate_fct(regisl_genenames_in4K, topgenes_IPC, branch_name='IPC',database_name='regulatory_islands')

oRG_overlap = p_EarlyLate_fct(regisl_genenames_in4K, topgenes_oRG, branch_name='oRG',database_name='regulatory_islands')

merged_df = pd.concat([ipc_overlap, oRG_overlap], axis=0)

In [ ]:
%matplotlib inline

In [ ]:
matrix = merged_df.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if matrix.shape[0] < 2:
    matrix = matrix.T


g = sns.heatmap(matrix, cmap='coolwarm', 
                annot=True, fmt='.2f',
                cbar_kws={'label': 'Percentage overlap'},
               xticklabels=['vRG-IPC', 'vRG-oRG'])


g.set_title('Overlap piNMF modules & genes in regulatory islands', fontsize=10)
g.set_xlabel('Branch')
g.set_ylabel('')

plt.show()

In [ ]:
matrix = merged_df.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if matrix.shape[0] < 2:
    matrix = matrix.T

plt.figure(figsize=(5, 5), dpi=300)

g = sns.heatmap(matrix, cmap='coolwarm', 
                annot=True, fmt='.2f',
                cbar_kws={'label': 'Percentage'},
               xticklabels=['vRG-IPC', 'vRG-oRG'])

g.set_title('', fontsize=8) #Gene overlap piNMF modules & regulatory islands
g.set_xlabel('Branch')
g.set_ylabel('Modules')

plt.show()

In [ ]:
merged_df.reset_index(inplace=True, drop=True)
merged_df

# Add Polioudakis 2019 data

# Top 100 genes in piNMF - Polioudakis

**oRG**

In [ ]:
pol19_dir = '/home/jovyan/jm_jlab/data_indNeuro/3.NMF_data/4k_oRG_polioudakis2019/4k_oRG_polioudakis2019/'

In [ ]:
pol19_topgenes_oRG = pd.read_csv(pol19_dir+'modules_to_oRG_topgenes_Polioudakis2019_4K_piNMF.tsv', sep='\t')

In [ ]:
for i in pol19_topgenes_oRG.columns:
    print(len(list(set(regisl_genenames_in4K) & set(pol19_topgenes_oRG[i]))))

In [ ]:
pol19_dir_ipc = '/home/jovyan/jm_jlab/data_indNeuro/3.NMF_data/4k_IPC_polioudakis2019/4k_IPC_polioudakis2019/'

In [ ]:
pol19_topgenes_IPC = pd.read_csv(pol19_dir_ipc+'modules_to_IPC_topgenes_Polioudakis2019_4K_piNMF.tsv', sep='\t')

In [ ]:
for i in pol19_topgenes_IPC.columns:
    print(len(list(set(regisl_genenames_in4K) & set(pol19_topgenes_IPC[i]))))

In [ ]:
pol19_ipc_overlap = p_EarlyLate_fct(regisl_genenames_in4K, pol19_topgenes_IPC, branch_name='vRG-IPC_Polioudakis19',database_name='regulatory_islands')

pol19_oRG_overlap = p_EarlyLate_fct(regisl_genenames_in4K, pol19_topgenes_oRG, branch_name='vRG-oRG_Polioudakis19',database_name='regulatory_islands')

merged_df2 = pd.concat([pol19_ipc_overlap, pol19_oRG_overlap], axis=0)

In [ ]:
%matplotlib inline

In [ ]:
matrix2 = merged_df2.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if matrix2.shape[0] < 2:
    matrix2 = matrix2.T

g = sns.heatmap(matrix2, cmap='coolwarm', 
                annot=True, fmt='.2f',
                cbar_kws={'label': 'Percentage'},
               xticklabels=['vRG-IPC', 'vRG-oRG'])

g.set_title('Gene overlap piNMF modules & regulatory islands', fontsize=8)
g.set_xlabel('')
g.set_ylabel('Modules')

plt.show()

In [ ]:
merged_df2.reset_index(inplace=True, drop=True)
merged_df2

In [ ]:
final_df = pd.concat([merged_df, merged_df2])

In [ ]:
matrix3 = final_df.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if matrix3.shape[0] < 2:
    matrix3 = matrix3.T

g = sns.heatmap(matrix3, cmap='coolwarm', 
                annot=True, fmt='.2f', 
                cbar_kws={'label': 'Percentage'},
               xticklabels=['vRG-IPC_Trevino21', 'vRG-oRG_Trevino21', 
                            'vRG-IPC_Polioudakis19', 'vRG-oRG_Polioudakis19'])


g.set_title('Gene overlap piNMF modules & regulatory islands', fontsize=8)
g.set_xlabel('')
g.set_xticklabels(g.get_xticklabels(), fontsize=8)
g.set_ylabel('Modules')

plt.show()

In [ ]:
ipc_deserts = p_EarlyLate_fct(ri_deserts_in4K, topgenes_IPC, branch_name='vRG-IPC',database_name='Deserts')

oRG_deserts = p_EarlyLate_fct(ri_deserts_in4K, topgenes_oRG, branch_name='vRG-oRG',database_name='Deserts')

ipc_possel = p_EarlyLate_fct(ri_possel_in4K, topgenes_IPC, branch_name='vRG-IPC',database_name='PosSel')

oRG_possel = p_EarlyLate_fct(ri_possel_in4K, topgenes_oRG, branch_name='vRG-oRG',database_name='PosSel')


In [ ]:
evo_sets = pd.concat([ipc_deserts,
                      oRG_deserts,
                      ipc_possel,
                      oRG_possel])


m_evosets = evo_sets.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if m_evosets.shape[0] < 2:
    m_evosets = m_evosets.T

g = sns.heatmap(m_evosets, cmap='coolwarm', 
                annot=True, fmt='.2f', 
                cbar_kws={'label': 'Percentage'},
                xticklabels=['vRG-IPC_deserts', 'vRG-oRG_deserts', 
             'vRG-IPC_PositiveSelection', 'vRG-oRG_PositiveSelection'])


g.set_title('Overlap piNMF modules & Deserts / Positive selection', fontsize=10)
g.set_xlabel('')
g.set_xticklabels(g.get_xticklabels(), fontsize=8)
g.set_ylabel('Modules')

plt.show()

In [ ]:
evo_sets.reset_index(inplace=True, drop=True)

In [ ]:
evo_sets

In [ ]:
# All dataframes
final_all = pd.concat([merged_df, 
                       merged_df2,
                      ipc_deserts,
                      oRG_deserts,
                      ipc_possel,
                      oRG_possel])


matrix4 = final_all.pivot(index='Module', columns=['Database', 'Branch'], values='Percentage overlap')

if matrix4.shape[0] < 2:
    matrix4 = matrix4.T

plt.figure(dpi=300)

g = sns.heatmap(matrix4, cmap='coolwarm', 
                annot=True, fmt='.2f', 
                cbar_kws={'label': 'Percentage'})

g.set_title('')
g.set_xlabel('')
g.set_xticklabels(g.get_xticklabels(), fontsize=8)
g.set_ylabel('')

plt.show()

In [ ]:
final_all.reset_index(inplace=True, drop=True)
final_all

In [ ]:
final_all.to_csv("/home/jovyan/jm_jlab/CBL_data/indirectNeurogenesis/nmf_data/Overlap_EvoSets.tsv", sep='\t',index=False)

# *Optional exploration of CellOracle Top TFs*

In [ ]:
tr21_links = co.load_hdf5(file_path="/home/jovyan/jm_jlab/data_indNeuro/1.GRN/milestones_pcw16_baggindridge.celloracle.links")

In [ ]:
# Selecting top 10 TFs for each cluster on eigenvector centrality

IPC_GRN = []

vRG_GRN = []

oRG_GRN = []

for cluster in ['vRG', 'oRG', 'IPC']:
    
    if cluster == 'vRG':
        res = tr21_links.merged_score[tr21_links.merged_score.cluster == cluster]
        vRG_GRN.append(res['eigenvector_centrality'].sort_values(ascending=False).head(10).index.tolist())
        vRG_GRN =[item for sublist in vRG_GRN for item in sublist]      
        
    if cluster == 'oRG':
        res = tr21_links.merged_score[tr21_links.merged_score.cluster == cluster]
        oRG_GRN.append(res['eigenvector_centrality'].sort_values(ascending=False).head(10).index.tolist())
        oRG_GRN =[item for sublist in oRG_GRN for item in sublist]                
        
    if cluster == 'IPC':
        res = tr21_links.merged_score[tr21_links.merged_score.cluster == cluster]
        IPC_GRN.append(res['eigenvector_centrality'].sort_values(ascending=False).head(10).index.tolist())
        IPC_GRN =[item for sublist in IPC_GRN for item in sublist]                


In [ ]:
d_vRG = []
d_oRG = []
d_IPC = []
for cluster in ['vRG', 'oRG', 'IPC']:
    l = []
    if cluster == 'vRG':
        for gene in vRG_GRN:
            l.append(tr21_links.filtered_links[cluster][tr21_links.filtered_links[cluster]['source'] == gene]['target'].tolist())
            l.append(gene)    
        d_vRG = [item for sublist in l for item in sublist] 

        
    if cluster == 'oRG':
        for gene in oRG_GRN:
            l.append(tr21_links.filtered_links[cluster][tr21_links.filtered_links[cluster]['source'] == gene]['target'].tolist())
            l.append(gene)
        d_oRG = [item for sublist in l for item in sublist]

    if cluster == 'IPC':
        for gene in IPC_GRN:
            l.append(tr21_links.filtered_links[cluster][tr21_links.filtered_links[cluster]['source'] == gene]['target'].tolist())
            l.append(gene)
        d_IPC = [item for sublist in l for item in sublist]                

In [ ]:
# Overlap to targets from top 10 TFs in each cluster
for list_of_genes in [d_vRG, d_oRG, d_IPC]:
    print(len(set(regisl_genenames_in4K) & set(list_of_genes))/(len(set(list_of_genes)))*100)